In [136]:
import os
import json
import time
from tqdm import tqdm
os.chdir("/Users/ruichenyang/projects/music_data/")

In [33]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import yaml
with open("config.yml", "rb") as f:
    config = yaml.safe_load(f)  

In [176]:
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=config['spotify_client_id'],
        client_secret=config['spotify_client_secret'],
        ))

In [4]:
def print_json(result):
    ""
    return print(json.dumps(result, indent=4))

### Get Artist Data Through Artist Name
    - use search query
    - take the most similar query (the first artist)

In [142]:
result = sp.search(q='whitney houston', type='artist')

In [143]:
artist = pd.DataFrame([[
    result['artists']['items'][0]['uri'],
    result['artists']['items'][0]['name'],
    result['artists']['items'][0]['popularity'],
    result['artists']['items'][0]['images'][0]['url'],
    ', '.join(result['artists']['items'][0]['genres']),
    result['artists']['items'][0]['followers']['total'],
    ]], columns=['artist_uri', 'artist_nm', 'artist_pop', 'artist_img', 'artist_genre', 'artist_num_followers'])

### Get Track Data Through Track Name

In [128]:
result = sp.search(q='i wanna dance with sombody', type='track')

In [129]:
#print_json(result['tracks']['items'][0])

In [193]:
#print_json(sp.tracks(track_uris[:5])['tracks'])

In [130]:
def get_tracks_from_query(result):
    ""
    tracks = pd.DataFrame()

    for i in range(len(result['tracks']['items'])):
        track = pd.DataFrame([[
            result['tracks']['items'][i]['uri'],
            result['tracks']['items'][i]['name'],
            result['tracks']['items'][i]['popularity'],
            result['tracks']['items'][i]['duration_ms'],
            len(result['tracks']['items'][i]['available_markets']), # could be binary features
            result['tracks']['items'][i]['external_urls']['spotify'],
            ', '.join([d['name'] for d in result['tracks']['items'][i]['artists']]),
            result['tracks']['items'][i]['album']['uri'], 
            result['tracks']['items'][i]['album']['name'],
            result['tracks']['items'][i]['album']['album_type'],
            result['tracks']['items'][i]['album']['images'][0]['url'],
            result['tracks']['items'][i]['album']['total_tracks'],
            result['tracks']['items'][i]['album']['release_date'],
            result['tracks']['items'][i]['album']['external_urls']['spotify'],
            ]], columns=['track_uri', 'track_nm', 'track_pop', 'track_runtime', 'track_num_countries', 'track_url', 'artist_nm', 'album_uri', 'album_nm', 'album_type', 'album_img', 'album_num_tracks', 'album_release_dt', 'album_url'])
        tracks = pd.concat([tracks, track], axis=0)

    return tracks

In [131]:
tracks = get_tracks_from_query(result)

while result['tracks']['next']:
    try:
        result = sp.next(result['tracks'])
        tracks = pd.concat([tracks, get_tracks_from_query(result)])
    except:
        print('end of result.')
        result = None
        pass


HTTP Error for GET to https://api.spotify.com/v1/search?query=i+wanna+dance+with+sombody&type=track&offset=1000&limit=10 with Params: {} returned 400 due to Bad request.


end of result.


TypeError: 'NoneType' object is not subscriptable

In [132]:
tracks[['track_uri', 'album_uri', 'track_nm', 'artist_nm', 'album_release_dt', 'track_url']].loc[lambda x: x['artist_nm'].str.lower().str.contains('whitney houston, p2j')].sort_values(by=['album_release_dt'], ascending=False)

,track_uri,album_uri,track_nm,artist_nm,album_release_dt,track_url
0,spotify:track:1uMmvZ8fmYUinSVbs5HTpK,spotify:album:00EjgohJGEYfe4vP35LVOv,I Wanna Dance With Somebody (Who Loves Me),"Whitney Houston, P2J",2022-12-16,https://open.spotify.com/track/1uMmvZ8fmYUinSVbs5HTpK


In [133]:
tracks['album_uri'].nunique()

912

In [146]:
album_uris = tracks['album_uri'].unique().tolist()
result = None

for b in tqdm(range(len(album_uris) // 10), desc='In Progress'):
    #print(f"in progress: {b*10}/{tracks['album_uri'].nunique()}")
    batch_uris = album_uris[b*10:min((b+1)*10, len(album_uris))]
    result = sp.albums(batch_uris)

    for a in result['albums']:
        tracks.loc[lambda x: x['album_uri'] == a['uri'], 'album_cpr_txt'] = a['copyrights'][0]['text'] if a['copyrights'] else ''
        tracks.loc[lambda x: x['album_uri'] == a['uri'], 'album_cpr_type'] = ', '.join([cpr['type'] for cpr in a['copyrights']]) if a['copyrights'] else ''
        tracks.loc[lambda x: x['album_uri'] == a['uri'], 'album_label'] = a['label'] if a['label'] else ''
    
    time.sleep(3)

In Progress: 100%|██████████| 91/91 [04:55<00:00,  3.25s/it]


In [147]:
tracks.head(2)

,track_uri,track_nm,track_pop,track_runtime,track_num_countries,track_url,artist_nm,album_uri,album_nm,album_type,album_img,album_num_tracks,album_release_dt,album_url,album_cpr_txt,album_cpr_type,album_label
0,spotify:track:2tUBqZG2AbRi7Q0BIrVrEj,I Wanna Dance with Somebody (Who Loves Me),80,291293,184,https://open.spotify.com/track/2tUBqZG2AbRi7Q0BIrVrEj,Whitney Houston,spotify:album:5Vdzprr5cOqXQo44eHeV7t,Whitney,album,https://i.scdn.co/image/ab67616d0000b273cc57e9b00b87dd0f6e868347,11,1987-06-02,https://open.spotify.com/album/5Vdzprr5cOqXQo44eHeV7t,(P) 1987 Arista Records LLC,P,Arista/Legacy
0,spotify:track:10BVvCsRJ7SxJmZEny4AVq,I Wanna Dance with Somebody (DJ Fluke Remix),33,210967,184,https://open.spotify.com/track/10BVvCsRJ7SxJmZEny4AVq,"DJ Femme, DJ Fluke, Hayley Teal",spotify:album:6dE2NwNuRjF4UYPQK15trb,I Wanna Dance with Somebody (DJ Fluke Remix),single,https://i.scdn.co/image/ab67616d0000b27365c997d78d278c30378d0205,1,2022-09-02,https://open.spotify.com/album/6dE2NwNuRjF4UYPQK15trb,2022 ATN Recordings,"C, P",ATN Recordings


In [148]:
artist.head(2)

,artist_uri,artist_nm,artist_pop,artist_img,artist_genre,artist_num_followers
0,spotify:artist:6XpaIBNiVzIetEPCWDvAFP,Whitney Houston,78,https://i.scdn.co/image/ab6761610000e5ebcd9f60ab57585bf3b77ecc51,"dance pop, urban contemporary",8697270


In [150]:
#sp.search('post malone')

In [154]:
#print_json(sp.artist_related_artists('spotify:artist:246dkjvS1zLTtiykXe5h60'))

In [158]:
#print_json(sp.artist_top_tracks('spotify:artist:246dkjvS1zLTtiykXe5h60', country='US'))

In [160]:
#print_json(sp.audio_analysis('spotify:track:2tUBqZG2AbRi7Q0BIrVrEj'))

In [164]:
print_json(sp.audio_features('spotify:track:10BVvCsRJ7SxJmZEny4AVq'))

[
    {
        "danceability": 0.74,
        "energy": 0.947,
        "key": 1,
        "loudness": -2.527,
        "mode": 1,
        "speechiness": 0.0447,
        "acousticness": 0.0113,
        "instrumentalness": 0.00631,
        "liveness": 0.0844,
        "valence": 0.197,
        "tempo": 123.975,
        "type": "audio_features",
        "id": "10BVvCsRJ7SxJmZEny4AVq",
        "uri": "spotify:track:10BVvCsRJ7SxJmZEny4AVq",
        "track_href": "https://api.spotify.com/v1/tracks/10BVvCsRJ7SxJmZEny4AVq",
        "analysis_url": "https://api.spotify.com/v1/audio-analysis/10BVvCsRJ7SxJmZEny4AVq",
        "duration_ms": 210968,
        "time_signature": 4
    }
]


In [173]:
track_uris = tracks['track_uri'].unique().tolist()
result = None

for b in tqdm(range(len(track_uris) // 10), desc='In Progress'):
    batch_uris = track_uris[b*10:min((b+1)*10, len(track_uris))]
    result = sp.audio_features(batch_uris)

    for t in result:
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_danceability']     = t['danceability']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_energy']           = t['energy']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_loudness']         = t['loudness']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_speechiness']      = t['speechiness']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_acousticness']     = t['acousticness']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_instrumentalness'] = t['instrumentalness']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_liveness']         = t['liveness']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_valence']          = t['valence']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_tsig']             = t['time_signature']
        tracks.loc[lambda x: x['track_uri'] == t['uri'], 'track_tempo']            = t['tempo']
    
    time.sleep(3)


In Progress: 100%|██████████| 98/98 [05:10<00:00,  3.17s/it]


In [174]:
tracks.head(2)

,track_uri,track_nm,track_pop,track_runtime,track_num_countries,track_url,artist_nm,album_uri,album_nm,album_type,album_img,album_num_tracks,album_release_dt,album_url,album_cpr_txt,album_cpr_type,album_label,track_danceability,track_energy,track_loudness,track_speechiness,track_acousticness,track_instrumentalness,track_liveness,track_valence,track_tsig,track_tempo
0,spotify:track:2tUBqZG2AbRi7Q0BIrVrEj,I Wanna Dance with Somebody (Who Loves Me),80,291293,184,https://open.spotify.com/track/2tUBqZG2AbRi7Q0BIrVrEj,Whitney Houston,spotify:album:5Vdzprr5cOqXQo44eHeV7t,Whitney,album,https://i.scdn.co/image/ab67616d0000b273cc57e9b00b87dd0f6e868347,11,1987-06-02,https://open.spotify.com/album/5Vdzprr5cOqXQo44eHeV7t,(P) 1987 Arista Records LLC,P,Arista/Legacy,0.709,0.824,-8.824,0.0453,0.2070,0.000307,0.0888,0.867,4.0,118.818
0,spotify:track:10BVvCsRJ7SxJmZEny4AVq,I Wanna Dance with Somebody (DJ Fluke Remix),33,210967,184,https://open.spotify.com/track/10BVvCsRJ7SxJmZEny4AVq,"DJ Femme, DJ Fluke, Hayley Teal",spotify:album:6dE2NwNuRjF4UYPQK15trb,I Wanna Dance with Somebody (DJ Fluke Remix),single,https://i.scdn.co/image/ab67616d0000b27365c997d78d278c30378d0205,1,2022-09-02,https://open.spotify.com/album/6dE2NwNuRjF4UYPQK15trb,2022 ATN Recordings,"C, P",ATN Recordings,0.740,0.947,-2.527,0.0447,0.0113,0.006310,0.0844,0.197,4.0,123.975
